# Text Preprocessing

In [1]:
import collections
import re
from d2l import torch as d2l

## Time Machine

In [2]:
#@save
# 将时间机器数据集的 URL 和 SHA-1 校验码存储到 DATA_HUB 字典中
d2l.DATA_HUB['time_machine'] = (
    d2l.DATA_URL + 'timemachine.txt',
    '090b5e7e70c295757f55df93cb0a180b9691891a'
)

# 定义函数 read_time_machine，用于将时间机器数据集加载到文本行的列表中
def read_time_machine():
    # 打开时间机器数据集文件，并将每一行的文本存储到列表 lines 中
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    # 对于每一行的文本，将其中的非字母字符替换为空格，并将其转换为小写字母
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

# 调用 read_time_machine 函数，将时间机器数据集加载到 lines 列表中
lines = read_time_machine()
# 打印文本总行数和第一行文本
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
# 打印第十行文本
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


## Tokenization

In [3]:
def tokenize(lines, token='word'):  #@save
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)

tokens = tokenize(lines, 'word')
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


## Vocabulary

In [4]:
class Vocab:
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        # 如果 tokens 为 None，则初始化为空列表
        if tokens is None:
            tokens = []
        # 如果 reserved_tokens 为 None，则初始化为空列表
        if reserved_tokens is None:
            reserved_tokens = []
        # 统计 tokens 中每个词元的出现频率
        counter = count_corpus(tokens)
        # 按照词元出现频率从高到低排序
        self._token_freqs = sorted(
            counter.items(), 
            key=lambda x: x[1],
            reverse=True
        )
        # 初始化词元索引列表，将未知词元的索引设为 0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        # 初始化词元到索引的映射字典
        self.token_to_idx = {
            token: idx for idx, token in enumerate(self.idx_to_token)
        }
        # 将出现频率大于等于 min_freq 的词元加入词表
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        # 如果 tokens 不是列表或元组，则返回对应词元的索引，
        # 如果不存在则返回未知词元的索引
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        # 如果 tokens 是列表或元组，则返回对应词元的索引列表
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        # 如果 indices 不是列表或元组，则返回对应索引的词元，
        # 如果不存在则返回未知词元
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        # 如果 indices 是列表或元组，则返回对应索引的词元列表
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):
    """统计词元的频率"""
    # 如果 tokens 是二维列表，则将其展平为一维列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    # 统计词元的出现频率
    return collections.Counter(tokens)

In [5]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [6]:
for i in [0, 10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])

文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


## 汇总

In [7]:
# 定义函数 load_corpus_time_machine，
# 用于加载时光机器数据集的词元索引列表和词表
def load_corpus_time_machine(max_tokens=-1):
    # 调用 read_time_machine 函数，将时间机器数据集加载到 lines 列表中
    lines = read_time_machine()
    # 对于每一行的文本，将其中的字符转换为词元，并将所有词元存储到列表 tokens 中
    tokens = tokenize(lines, 'char')
    # 初始化词表 vocab，将 tokens 中的词元加入词表
    vocab = Vocab(tokens)
    # 将 tokens 中的词元转换为对应的索引，并将所有索引存储到列表 corpus 中
    corpus = [vocab[token] for line in tokens for token in line]
    # 如果 max_tokens 大于 0，则只保留前 max_tokens 个词元
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    # 返回词元索引列表 corpus 和词表 vocab
    return corpus, vocab

# 调用 load_corpus_time_machine 函数，将时光机器数据集加载到 corpus 列表中，并将词表加载到 vocab 中
corpus, vocab = load_corpus_time_machine()
# 打印词元数量和词表大小
print(len(corpus), len(vocab))

170580 28
